# Micrograph analysis tutorial

This is a tutorial on how to use SimpliPyTEM to simplify the analysis of electron microscopy images. This includes basic functions including filtering, converting the file to 8-bit, looking at metadata and more.

## Import dependancies
The reason it says \<Figure size...\> is because I run plt.gray() which makes plt.imshow() show data in a grayscale image by default, this can be ignored. 

In [1]:
from SimpliPyTEM.Micrograph_class import * 

<Figure size 640x480 with 0 Axes>

## Initialise the Micrograph object and open a micrograph

The Micrograph object is the heart of SimpliPyTEM for images, this contains  the methods, data and metadata available with SimpliPyTEM. To use, initialise the object and then run one of the methods to open a file - open_dm for digital micrograph files, open_mrc for mrc files and open_tif for tif files. 

Note that if a video (or dose fractionation) dm file is provided, the micrograph object will automatically sum the frames, opening it as an image. To load video files as videos, use the MicroVideo object. 

An example image is included with this tutorial, and at times will be commented on itself, however you are very welcome to use your own image, hopefully very similar results will be seen.


In [ ]:
#initialise object called im, this currently is an empty object

micrograph = Micrograph()

#open micrograph

micrograph.open_dm('A1_Tribloc-250000X-0006.dm4')




### Lets familiarise ourselves with the Micrograph object. 
This is a python object which contains the image data, metadata, and methods to handle the image. The most important thing is to access the image, this can be done through the command below:





In [ ]:
micrograph.image

This output shows the image data, this is kept as a 2D array with all the pixel values that make up an image. To see the shape of the image you can use the .shape property

In [ ]:
print(micrograph.image.shape)

# or, to make it easier:

print(micrograph.shape)

Well this is a start, but who wants to look at the actual values in the array? lets see what the image actually looks like, this can be done using matplotlib.pyplot.imshow() or through a built in function that calls this. 


In [ ]:
micrograph.imshow()

In [ ]:
plt.imshow(micrograph.image)
plt.show()

You'll see that the results are different, this is because I have specified the size plotted in the object. There are lots of options to plot images with matplotlib and I recommend looking into them if you require clearer observation in an iPython environment. I generally use simple functions to plot here, and save the output for display purposes though.

## Other properties and metadata

Now we have the most important thing sorted, the image. However, theres lots of other useful information stored in a Micrograph object, this is particularly true when opening a digital micrograph file. We can access this as follows: 

### What is the pixel size of my image? 

In [ ]:
print(micrograph.pixelSize)

well that could mean anything, what is the unit? 

In [ ]:
print(micrograph.pixelUnit)

Oh cool, what magnification does that mean it was? 

In [ ]:
micrograph.get_mag()


In [ ]:
'''This function prints the magnifications, but also returns them (as floating point numbers) 
    so you can have them for further use'''
#indicated_mag, actual_mag = micrograph.get_mag()
#print(indicated_mag, actual_mag)

How about other imaging conditions? Voltage? Exposure? Acquisition date and time? 

In [ ]:
voltage = micrograph.get_voltage()
print(voltage)

exposure = micrograph.get_exposure()
print('Exposure time: ',exposure,'seconds')

micrograph.get_date_time()

Hold on a second! Why do some of these require brackets (eg. .get_voltage(), but some don't e.g. .pixelSize? 

*It seemed to me writing this some properties are required often so should be saved as default, others my only be required at specific occasions and can be kept hidden. The good thing is, once you have run these 'getter' functions, the data is printed, returned and saved to the object. This final point means once you have run the getters once, you can just try this:*

In [ ]:
print(micrograph.voltage)
print(micrograph.exposure)
print(micrograph.AqDate, micrograph.AqTime)
print(micrograph.indicated_mag, micrograph.actual_mag)

Ok this is good, but I need to check a very specific setting, is this saved anywhere? 

*Well I'm not making any promises but if the setting is saved in the image metadata it should be available if you are willing to look for it. Try here:*


In [ ]:
#micrograph.show_metadata()

#This is wont appear because it would take up several pages.

Well it's there, but I want to find this on a whole bunch of files, is there a way to avoid looking through the list each time? 

*Yes, the metadata is saved as a dictionary in Micrograph.metadata_tags, find the name of the parameter from above and access it as follows:*

In [ ]:
micrograph.metadata_tags['.ImageList.2.ImageTags.Acquisition.Device.Name']

*You can even save it to the object as follows:*

In [ ]:
#micrograph.Detector_model = micrograph.metadata_tags['.ImageList.2.ImageTags.Acquisition.Device.Source Model']
#print(micrograph.Detector_model)

*if theres a setting you want regularly, you might consider making a getter function within the Micrograph_class.py file, search for the get_voltage() function for an example of how to do this*

## Micrograph Editing

These are some basic functions to edit the micrograph. 


### Convert to 8 bit 
 
Micrographs generally come as 32-bit or 16-bit images, this means the pixel values can between 0 and (2^16-1) or (2^32-1), with 32-bit having the added complication of having floating point numbers (decimals) rather than just integers. This allows much more data to be kept in an individual image, and much more contrast available in the individual image. However, most uses for images (including simple viewing with the majority of programs) require 8-bit images. Therefore, we often need to convert to 8-bit, so there is a simple method to do so provided in the Micrograph object:

In [ ]:
micrograph8bit = micrograph.convert_to_8bit()

We can check if it works by checking the pixel values of the image:

In [ ]:
print('Original max = {} :  8bit max =  {}'.format(micrograph.image.max(), micrograph8bit.image.max()))
print('Original min = {} :  8bit min =  {}'.format(micrograph.image.min(), micrograph8bit.image.min()))
print('Original mean= {} :  8bit mean=  {}'.format(micrograph.image.mean(), micrograph8bit.image.mean()))

We can see this therefore scales the image between 0 and 255 (2^8 - 1) and makes the values integers, thus making it an 8-bit image
Lets just check to see what effect it has on the image: 

In [ ]:
micrograph.show_pair(micrograph8bit.image, title1='Original', title2='8-bit')

No obvious effect?! This is for 2 reasons: 
- Firstly, we cant see all the contrast contained in the image, for viewing purposes 8bit is as good as we need, the additional contrast from higher bit images only helps when contrast is edited/enhanced.
- Secondly, I have plotted this with plt.imshow from matplotlib. This automatically scales the image to show the minimum value in the image as black and the max as white.

Lets try plotting it with a defined max and min value (which is what we would see when saving the image):


In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,10))
ax[0].imshow(micrograph.image, vmax=255, vmin=0)
ax[0].set_title('Original')
ax[1].imshow(micrograph8bit.image, vmax=255, vmin=0)
ax[1].set_title('8-bit')
plt.show()

Now we can see the difference. 

Converting to 8-bit will therefore enhance the contrast by scaling it between the max and min values of an 8-bit image. It is also required to perform many functions, many of these will convert automatically when using the micrograph object, but its good to keep this in mind.  

## Improving  contrast



Contrast can be enhanced using Three different functions: micrograph.clip_contrast(), micrograph.enhance_contrast() and micrograph.eqHist(), the latter two both make use of [functions from openCV](https://docs.opencv.org/3.4/d3/dc1/tutorial_basic_linear_transform.html).

My favourite however, is clip contrast because this will give the most consistent results. 

### Clip Contrast

A digital image is an array of pixels with values between a maximum and minimum value, which in the case of 8 bit images are 0 and 255. The number of pixels with each value can be seen in a histogram, a built in function for this is included in the micrograph object:


In [ ]:
micrograph8bit.plot_histogram()

We can see in this histogram that the image does not include the full range of pixel intensities, and instead is mainly in the range of 50-150. This means the image will look 'flat' or 'grey' containing few examples of true black or true white in the image. we can see this if we plot it with defined values:

In [ ]:
plt.imshow(micrograph8bit.image, vmax=255, vmin=0)
plt.show()

See? We have no black or white pixels, instead we just see shades of grey. 

To improve this, we can just pull the histogram so that the pixel range represented by the peak in the histogram, 50-150 are spread over the range 0 to 255. This is where the clip_contrast function comes in:

In [ ]:
micrograph8bit_enhanced = micrograph8bit.clip_contrast(maxvalue=150, minvalue=50)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,10))
ax[0].imshow(micrograph8bit_enhanced.image, vmax=255, vmin=0)
ax[1].hist(micrograph8bit_enhanced.image.ravel(), 255,(0,255))
plt.show()

Looks a bit better right!

But what if we don't want to have to manually choose the range to clip the contrast? Well theres a value which can be used instead of the max and min value - the saturation. 

The saturation is the percentage of pixels in the original video that will be set to 0 or 255 (and are therefore 'saturated'). Therefore this is a value which increases with the contrast. This is by default set to 0.5 which is quite a strong contrast, however the value can be set manually.

In [ ]:
micrograph8bit_clipped05 = micrograph8bit.clip_contrast()

fig, ax = plt.subplots(1,2, figsize=(20,10))
ax[0].imshow(micrograph8bit_clipped05.image, vmax=255, vmin=0)
ax[1].hist(micrograph8bit_clipped05.image.ravel(), 255,(0,255))
plt.show()

That gives a stronger effect! 

Lets look at how the saturation changes the contrast:

In [ ]:
saturations = [0.1, 0.5,1, 5,10,25]

fig, ax = plt.subplots(2,3, figsize=(30,15
                                    
                                    ))
for i in range(len(saturations)):
    sat = saturations[i]
    clipped = micrograph8bit.clip_contrast(saturation = sat)
    j=0
    if i>2:
        j=1
        i=i-3
    ax[j, i].imshow(clipped.image)
    ax[j, i].set_title('Saturation = {}'.format(sat), fontsize=30)

plt.show()


### Enhance_contrast 

Takes either 2 or 3 inputted values, these are alpha, beta and gamma (optional). 
- Alpha controls contrast, and is normally used in ranges of 1-3. 
- Beta is a brightness control and simply adds this value to every pixel in the image. This will only have a major affect with 8-bit images and is best used with values between -100 and +100 (negative is making the image darker. 
- Gamma controls contrast in a non-linear way. Values between 0-1 makes the image brighter (particularly dark pixels) whilst values >1 darken the image. Gamma requires an 8bit image and if included will automatically convert the image to 8bit.


For more information on these I recommend the link above.

The default is to run with alpha=1.5, beta=0 and no gamma. This is difficult to automate, and so its best to choose settings manually. 


In [ ]:
micrograph_alpha2 = micrograph8bit.enhance_contrast(alpha=2)
micrograph8bit.show_pair(micrograph_alpha2.image, title1='Original', title2='Alpha=2')


In [ ]:
micrograph_beta50 = micrograph8bit.enhance_contrast(alpha=1, beta=50)
micrograph8bit.show_pair(micrograph_beta50.image, title1='Original', title2='Beta=50')

In [ ]:
micrograph_gamma05 = micrograph8bit.enhance_contrast(alpha=1, beta=0, gamma=0.4)
micrograph8bit.show_pair(micrograph_gamma05.image, title1='Original', title2='Gamma=0.4')

### Histogram equalisation 

Histogram equalisation is a method to ensure the pixel values spread the possible values much evenly, flattening the histogram of pixel values. This process can be very effective at enhancing features, particularly with in images with a very dark patch or a very bright patch, the contrast in the midtones can be massively enhanced. 

This again only works on 8bit images, and thus the method in the micrograph object automatically converts it to 8bit using the function detailed above. 

This implementation also uses openCV and more information on the process can be [found here](https://docs.opencv.org/3.4/d4/d1b/tutorial_histogram_equalization.html). 

In [ ]:
micrographHistEQ = micrograph.eqHist()
micrograph.show_pair(micrographHistEQ.image, title1='Original', title2='Histogram equalised')

## Micrograph Filters! 

Now we are getting to the fun bit. Micrographs can look a bit rubbish as raw files, lets see if we can use the micrograph object to improve it a bit! 

These filters have been implemented to make them as simple as possible to use. I encourage users to familiarise yourselves with the filter and the code behind it (see Image filters section of Micrograph_class.py), as there may be settings which I have not made available for simplicity's sake but could be useful for you. However, for normal analysis, these should be perfectly effective. 

In SimpliPyTEM, filters return a new micrograph object with a filtered image, this allows trying different filters/parameters without risking overwriting the image.

First, lets see how it looks already:

In [ ]:
micrograph.imshow(title='Original image')

Looks quite noisy, lets try to filter it a bit...

### Gaussian filter

This is one of the most common image filters and is used to smooth an image. Here I use OpenCV's image function within the Micrograph_class, keeping only one setting: the kernal, this needs to be an odd integer. A bigger kernal leads to a stronger blurring effect.

In [ ]:
#this is the syntax for image filters:
MicrographGaussian = micrograph.gaussian_filter(7)


So here I've done a gaussian filter of the micrograph, which returns a new micrograph object that has been gaussian filtered. The kernal is defined by the '7' - this means a 7px by 7px kernal. 7Px is a large filter 

In [ ]:
MicrographGaussian.show_pair(micrograph.image, title1='Gaussian filter', title2='Original') #
# this is a built in function to view a pair of images side by side using plt.subplots 
# you this can be improved by making your own function, but this works for a quick check
# here the first 



### Median Filter

This is another common filter, and can effectively reduce 'salt-and-pepper' noise (anomalously low or high pixel values), this replaces each pixel with the median of the pixels around it in an n\*n square, where n is the kernal size. Again, the kernal size has to be odd, and a larger number gives a stonger effect. 

In [ ]:

MicrographMedian = micrograph.median_filter(3)
MicrographMedian.show_pair(micrograph.image, title1='Median filter',title2='Original' )

### Weiner filter

This is a more complex filter, which can filter out noise from corrupted signal. It can be effective for TEM images. This is implemented using the scipy.signal library.

In [ ]:
micrographWeiner = micrograph.weiner_filter(5)
micrographWeiner.show_pair(micrograph.image, title1='Median filter', title2 = 'Original')

### Low pass filtering

This filters the image based on the fourier transform of the image, allowing one to filter it to a required resolution. While this generally produces patterned artefacts, it can greatly enhance the contrast while sacrificing the higher resolution features. I have found this to be very effective at enhancing large, low contrast features. 

Assuming your micrograph object has a pixel size defined, the filter works by removing any features smaller than the size you input as a parameter (the unit is the same as the pixelsize). Therefore, a larger number yields a stronger filter. Beware because if it is too large, you won't see any features. Effective filter sizes depends on features and magnification, but maybe start with something like 5 and tune it to your needs.

If your micrograph is missing a pixelsize (maybe because you are not using a micrograph file, but a regular tif), then the size input will be the radius of a circle kept in the power spectrum. The result of this is the input number does the inverse - a smaller number leads to stronger filter. In this case, much larger numbers will be needed, so maybe start with 50 and from there. 

In [ ]:
micrographLowPass_5nm = micrograph.low_pass_filter(5) #filtering to 5nm
micrographLowPass_5nm.show_pair(micrograph.image, title1='5nm Low pass Filter', title2='Original')

In [ ]:
micrographLowPass_1nm = micrograph.low_pass_filter(1) #filtering to 1nm
micrographLowPass_10nm = micrograph.low_pass_filter(10)
micrographLowPass_1nm.show_pair(micrographLowPass_10nm.image, title1='1nm Low pass Filter', title2='10nm Low Pass Filter')

Here the 5nm LP filter greatly enhances the fibre in the image, while the 1nm shows a much more subtle, but still noticable improvement. 10nm is too strong however, and only artefacts remain.  

### Non-local means filter

This is a filter implemented using the openCV library, and [their description of the process](https://docs.opencv.org/3.4/d5/d69/tutorial_py_non_local_means.html) is pretty good. It denoises by comparing and averaging over different areas of the image. This can be effective, even if it does not appear so in this example. 

There are a few parameters allowed by openCV but I have only included one in this implementation - h, h controls the strength of the filter, with a bigger number leading to better noise removal but also more removal of the image. 

I don't find this filter amazing but I'm sure in certain circumstances it will be effective.

In [ ]:
micrographNLM = micrograph.nlm_filter(20)
micrographNLM.show_pair(micrograph.image, title1='Non-Local means filter', title2='Original')

## Add a scalebar

Every microscopist knows an micrograph means nothing without a scalebar, so I've made the process very easy. With a single command, it will choose a scalebar length, position and color (either black or white depending on contrast in local area).

I recommend adding the scalebar as a final step to the analysis process, because the scale bar perminently changes the pixel values to the max or min value, which will mess with any contrast enhancement (particularly scaling the image to the max and min). Also the filters will blur the scalebar. 

Unlike all the other adjustments, this doesnt return a new image, but will simply modify the image present, this is because it should be the final step and there are no real parameters to adjust to get it right. If you want to keep the original I recommend using deepcopy() to copy the micrograph object, as I have below. 


In [ ]:

micrographSB = micrograph.make_scalebar()
micrographSB.imshow()

Oh no! The scalebar has messed up the contrast shown. This is because the image is still a 32-bit image and not scaled between 0 and 255. imshow scales the image between the image max and min. The minimum pixel value was previously ~50, now it is zero. To fix this, convert to 8bit first:


In [ ]:
micrograph8bit=micrograph.convert_to_8bit()
micrograph8bit = micrograph8bit.make_scalebar()
micrograph8bit.imshow()

## Saving the image

The image can be easily saved from the micrograph object using the method write_image(). This can be used with no input at all, and will create an .jpg image named based on the original filename in the directory you are currently in. You can also specify the name of the output file by adding an input. 

Ending the name with .tif will save it as a tif file, this will by default be a 32-bit tif file unless you have previously converted the image to 8bit, and will have the pixel size included as a tag. 

You can specify a directory to output the files in with the keyword argument outdir='Directory'. This will create the new directory if it doesn't already exist. 

In [ ]:
micrograph.write_image()

In [ ]:
micrograph8bit.write_image('Tutorial_image.tif',outdir='images')

# Automating Pipelines

One big advantage of using python is how easy it is to automate repeat analysis with different files. This can be easily achived using a simple for loop:

Firstly collect the digital micrograph files (e.g. by using os.listdir to collect all the files in the directory, and list comprehension to find only the .dm* files):

In [ ]:
dm_files = [x for x in os.listdir('.') if x[-4:-1]=='.dm']

In [ ]:
for file in dm_files:
    print(file)
    micrograph = Micrograph()
    micrograph.open_dm(file)
    micrograph = micrograph.median_filter()
    micrograph = micrograph.convert_to_8bit()
    micrograph = micrograph.make_scalebar()
    micrograph.imshow()

To make this even easier, I have included a function in the Micrograph_class.py file which will do this in a single function: 


In [ ]:
for file in dm_files:
    default_image_pipeline(file) #There are more settings in this: medfilter=3, gaussfilter=0, scalebar=True, texton = True, xybin=2, color='Auto',**kwargs)

## Getting more information

Need help? You can always try the help() method, this shows the details of the function as annotated:

In [ ]:
help(micrograph)

In [ ]:
help(micrograph.make_scalebar)

In [ ]:
help(micrograph.gaussian_filter)

## Any more questions? 
Try raising an issue on github, or email me (Gabriel Ing) at ucbtgrb@ucl.ac.uk for more help! 